# Training and Converting an LSTM Model from PyTorch to Tensorflow Lite for Micro


## Training in PyTorch

Let's train a hand-written character recognition model with the MNIST dataset which uses LSTM models.

Start off with importing the dataset:

In [1]:
import torch
import torchvision
from torchvision import transforms

"""
Each data point is of form [PIL.Image, class] where the class is a number 0-9 of the
respective digit. The image is a single channel image with values from 0 to 1.
"""
def mnist_data(train: bool) -> torchvision.datasets.MNIST:
    return torchvision.datasets.MNIST(
        './mnist/',
        download=True,
        train=train, 
        transform = transforms.Compose([
            transforms.ToTensor()
        ])
)

# iterator returning Tensors of type [batch (10), channels (1), height (28), width (28)]
data_loaders = {
    "train": torch.utils.data.DataLoader(
        mnist_data(True),
        batch_size=100,
        shuffle=True,
        num_workers=4
    ),
    "test": torch.utils.data.DataLoader(
        mnist_data(False),
        batch_size=100,
        shuffle=True,
        num_workers=4
    ),}

### Define Model

In [24]:
class MNIST_LSTM(torch.nn.Module):
    def __init__(self, input_size: int, hidden_size: int, num_layers: int = 1):
        super(MNIST_LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.input_size = input_size
        
        self.lstm = torch.nn.LSTM(
            input_size = input_size,
            hidden_size = hidden_size,
            num_layers = num_layers,
            batch_first = True
        )
        self.dropout = torch.nn.Dropout(0.2)
        self.fc = torch.nn.Linear(hidden_size, 10)
        
    def forward(self, x):
        
        # Remove batch parameter
        x = x.reshape(-1, self.input_size, self.input_size)

        # Set initial hidden and cell states 
        hidden_initial = torch.zeros(self.num_layers, x.size(0), self.hidden_size) 
        cell_states_initial = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        lstm_out, hidden = self.lstm(x, (hidden_initial, cell_states_initial))
        
        dropout_out = self.dropout(lstm_out)
        
        fc_out = self.fc(dropout_out[:, -1, :])
        return fc_out
       

### Training Loop

In [26]:
input_size=28

# instantiate model
model = MNIST_LSTM(input_size, hidden_size=128)

loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [27]:
def train(num_epochs: int, model: MNIST_LSTM, loaders):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs}")
        for i, (images, labels) in enumerate(loaders['train']):
            outputs = model(images)
            loss = loss_func(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    print("Ended epoch: ")

In [28]:
train(num_epochs=3, model=model, loaders=data_loaders)

Epoch 0/3
Epoch 1/3
Epoch 2/3
Ended epoch: 


### Check that the model works

In [170]:
# Load one batch to test
images, labels = next(iter(data_loaders["test"]))

results = model(images).tolist()
processed_results = list(map(lambda res: res.index(max(res)), results))

# show first ten images and their calculated labels
for idx in range(10):
    display(torchvision.transforms.ToPILImage()(images[idx]))
    print(f"{processed_results[idx]}\n")

3



9



7



5



0



1



0



5



5



0



### Check Model Accuracy

In [30]:
# Test the model
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    
    for images, labels in data_loaders['test']:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total = total + labels.size(0)
        
        correct = correct + (predicted == labels).sum().item()
    
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 98.07 %


## Convert to ONNX

Now we can convert this to ONNX using PyTorch's built-in exporter. 

To do this, we need to provide a sample input. This sample input is passed as input to the model and the trace generated will be used to construct the ONNX model.

In [171]:
sample_input, sample_labels = next(iter(data_loaders["test"]))

onnx_filename = "handwriting.onnx"

torch.onnx.export(
    model,
    sample_input,
    onnx_filename,
    input_names= ["images"],
    output_names=["number"]
)

## Check ONNX
Let's just check that the ONNX conversion worked before we continue.

In [172]:
import onnx

# Load the ONNX model
onnx_model = onnx.load(onnx_filename)

# Check that the model is well formed
onnx.checker.check_model(onnx_model)

In [175]:
import numpy
import onnxruntime as ort

ort_session = ort.InferenceSession(onnx_filename)

outputs = ort_session.run(
    None,
    {"images": sample_input.numpy()},
)
onnx_processed_results = [numpy.argmax(res) for res in outputs[0]]

# show first ten images and their calculated labels
for idx in range(10):
    display(torchvision.transforms.ToPILImage()(sample_input[idx]))
    print(f"{onnx_processed_results[idx]}\n")


7



0



3



7



0



8



2



1



6



7



## ONNX To TFlite

Our ONNX model seems to be working correctly, so let's continue to convert our model into a Keras Model. From there, we can convert it into a TFLite file.

### ONNX to Keras

Here we use a library by [@PINTO0309](https://github.com/PINTO0309/) to convert our model into a Keras model. For full documentation, see: https://github.com/PINTO0309/onnx2tf

#### Important note about Parameter Replacement

`onnx2tf` converts PyTorch Tensors of shape BCHW into the form BHWC, as is convention for TensorFlow. It usually manages this fine, however, if it gets the shapes wrong, you can use the `param_replacement_file` to reshape an operator's input's our outputs to correct it. 

For example in this example, the `squeeze` after the LSTM operation was changed to operate on the 3rd dimension, but the output of the LSTM output was not updated to match; therefore, we add a `post_process_transform` to the LSTM operator to make it work. For more details, see: https://github.com/PINTO0309/onnx2tf#parameter-replacement

In [145]:
import onnx2tf

keras_model = onnx2tf.convert(
    input_onnx_file_path=onnx_filename,
    output_folder_path="mnist.tf",
    copy_onnx_input_output_names_to_tflite=True,
    non_verbose=True,
    param_replacement_file="param_replacement.json"
)

#### Check Keras Accuracy

Now that we have a Keras model, let's check its accuracy. If all went well, it should not have changed significantly from the original model's accuracy.

In [ ]:
keras_correct = 0
keras_total = 0

for test_images, test_labels in data_loaders['test']:
    #transpose the input_batch into BHWC order for tensorflow
    tf_input_data = np.transpose(test_images.numpy(), [0, 2, 3, 1])

    keras_output_data = keras_model(tf_input_data)
    processed_keras_output = [numpy.argmax(res) for res in keras_output_data]
    
    keras_results = zip(processed_keras_output, test_labels)
    keras_batch_correct = sum([1 if output == label.item() else 0 for output, label in keras_results])
        
    keras_correct = keras_correct + keras_batch_correct
    keras_total = keras_total + test_labels.size(0)


print(f"Test Accuracy of the keras model on the {keras_total} test images: {100 * keras_correct / keras_total} % ({keras_correct}/{keras_total})")

### Keras to TensorFlow Lite
Now let's quantise our model to int8 and export it to a TensorFlow Lite flatbuffer.


#### Representative Dataset

To convert a model into to a TFLite flatbuffer, a representative dataset is required to help in quantisation. Refer to [Converting a keras model into an xcore optimised tflite model for more details on this.](https://colab.research.google.com/github/xmos/ai_tools/blob/develop/docs/notebooks/keras_to_xcore.ipynb)


In [149]:
def representative_dataset():
    for test_images, _ in data_loaders['test']:
        tf_batch = np.transpose(test_images.numpy(), [0, 2, 3, 1])
        yield [tf_batch]

In [150]:
# Now do the conversion to int8
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.float32 
converter.inference_output_type = tf.float32

tflite_int8_model = converter.convert()

# Save the model.
tflite_int8_model_path = 'handwriting.tflite'
with open(tflite_int8_model_path, 'wb') as f:
  f.write(tflite_int8_model)

fully_quantize: 0, inference_type: 6, input_inference_type: FLOAT32, output_inference_type: FLOAT32


#### Check Accuracy of Converted Model

In [177]:
tfl_interpreter = tf.lite.Interpreter(model_path=tflite_int8_model_path)
tfl_interpreter.allocate_tensors()

tfl_input_details = tfl_interpreter.get_input_details()
tfl_output_details = tfl_interpreter.get_output_details()

tfl_correct = 0
tfl_total = 0

for test_images, test_labels in data_loaders['test']:
    #transpose the input_batch into BHWC order for tensorflow
    tfl_input_data = np.transpose(test_images.numpy(), [0, 2, 3, 1])

    tfl_interpreter.set_tensor(tfl_input_details[0]['index'], tfl_input_data)
    tfl_interpreter.invoke()

    tfl_output_data = tfl_interpreter.get_tensor(tfl_output_details[0]['index'])

    tfl_processed_results = [numpy.argmax(res) for res in tfl_output_data]
    
    tfl_results = zip(tfl_processed_results, test_labels)
    tfl_batch_correct = sum([1 if output == label.item() else 0 for output, label in tfl_results])
        
    tfl_correct = tfl_correct + tfl_batch_correct
    tfl_total = tfl_total + test_labels.size(0)

print(f"Test Accuracy of the TensorFlow Lite for Micro model on the {tfl_total} test images: {100 * tfl_correct / tfl_total} % ({tfl_correct}/{tfl_total})")

Test Accuracy of the TensorFlow Lite for Micro model on the 10000 test images: 97.98 % (9798/10000)


### Check operator counts of converted model
Let us take a look at the operator counts inside the converted model. Ideally, we want the same accuracy with minimal operators. This uses a helper function defined in ../utils, but this step is not necessary to convert the model.

In [179]:
import sys
import os 

# allow importing helper functions from local module
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import utils
utils.print_operator_counts(tflite_int8_model)

OPERATOR              COUNT
-------------------- ------
quantize                  6
reshape                   4
transpose                 2
dequantize                5
while                     1
gather                    2
fully_connected           3
less                      1
add                       8
split                     1
logistic                  3
mul                       3
tanh                      2
concatenation             3
slice                     2
-------------------- ------
TOTAL                    46
-------------------- ------
